In [ ]:
"""            
            """

In [27]:
from openai import OpenAI
from dotenv import load_dotenv
import os 
import copy
import numpy as np 
import sqlite3
import pandas as pd 

load_dotenv()

def get_completion(n_options):
    client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))

    response = client.responses.create(
        model="gpt-4.1-nano",
        input=f"""
            Write a string representation of a Python list where each element is a title of a popular Wikipedia article.

            The list should have {n_options} elements.

            The list should be articles that are interesting choices, but are well-known enough that they are possible to 
            guess as part of a challenging guessing game.

            Be unpredictable! This query will be ran many times and each time it should have a different result.

            Example output:

            ["Nintendo 3DS", "Dopamine", "Chinese language", "Bishop", "Cross-dressing", "Arnold Schwarzenegger", "Water tower", "Bouldering", "Wax", "Flag of Canada"]
        """
    )
    return response.output_text

def execute_sql(query):
    conn = sqlite3.Connection('data/data.db')
    data = pd.read_sql(query, conn).to_dict(orient='records')
    conn.close()
    return data

def get_matches(title):
    try:
        return execute_sql(f"""
            select article_id, title, count
            from articles 
            where clean_title like '%{title}%'
            order by count desc
            limit 1
        """)
    except Exception as e:
        print(e)
        print(f"Failed on: {title}")
        return []

In [35]:
import json

targets = set()
n = 100

for i in range(5):
    print(i)
    try:
        ai_choices = get_completion(n)
        ai_choices = ai_choices.strip().lower().replace(r"`", '').replace('python', '').strip()
        ai_choices_obj = eval(ai_choices)
        ai_choices_clean = [x.lower().strip() for x in ai_choices_obj]
        for ai_choice in ai_choices_clean:
            matches = get_matches(ai_choice.replace("'", ""))
            if not matches:
                continue
            targets.add(tuple([x for x in matches.pop().values()]))
    except Exception as e:
        print(e)

# Write targets out to disk.
with open('data/ai_targets.jsonl', 'a', encoding='utf-8') as file:
    for row in list(targets):
        file.write(json.dumps({'article_id': row[0], 'count': row[2]}) + '\n')